In [473]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Chargement des données

file_path = 'C:/Users/benab/OneDrive/Bureau/ML-JEE/enhanced_investment_data.csv'
data = pd.read_csv(file_path,encoding='ISO-8859-1')

# Afficher un aperçu des données
print("Aperçu des premières lignes du dataset :")
print(data.head())

Aperçu des premières lignes du dataset :
                              Nom  Genre        Ville  Age    Revenu  \
0  Constance Olivier Le LemaÃ®tre  Femme        Paris   48  91061.68   
1                    AdÃ¨le Duval  Homme  Montpellier   60  95493.18   
2         Simone Masson de Collin  Femme         Nice   40  84344.87   
3           Gabrielle Pichon-Paul  Femme         Lyon   44  59332.79   
4               Martin Le Chauvin  Femme       Nantes   40  75818.07   

  Tolérance au Risque   Historique d'Investissement   Objectif Financier  \
0               Élevé          Actions, Obligations             Retraite   
1               Moyen              ETF, Fundraising  Épargne de sécurité   
2               Élevé       Obligations, Immobilier             Retraite   
3               Moyen          Actions, Obligations             Retraite   
4               Élevé  Fundraising, ETF, Immobilier     Achat immobilier   

  Secteur Préféré Fréquence d'Investissement Domaine Recommandé  
0  

In [474]:
# 1. Vérification des informations de base sur le dataset
print("\nInformations générales sur le dataset :")
print(data.info())


Informations générales sur le dataset :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Nom                          1000 non-null   object 
 1   Genre                        1000 non-null   object 
 2   Ville                        1000 non-null   object 
 3   Age                          1000 non-null   int64  
 4   Revenu                       1000 non-null   float64
 5   Tolérance au Risque          1000 non-null   object 
 6   Historique d'Investissement  1000 non-null   object 
 7   Objectif Financier           1000 non-null   object 
 8   Secteur Préféré              1000 non-null   object 
 9   Fréquence d'Investissement   1000 non-null   object 
 10  Domaine Recommandé           1000 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 86.1+ KB
None


In [475]:
# 2. Vérification des valeurs manquantes
print("\nValeurs manquantes par colonne :")
print(data.isnull().sum())


Valeurs manquantes par colonne :
Nom                            0
Genre                          0
Ville                          0
Age                            0
Revenu                         0
Tolérance au Risque            0
Historique d'Investissement    0
Objectif Financier             0
Secteur Préféré                0
Fréquence d'Investissement     0
Domaine Recommandé             0
dtype: int64


In [476]:
# 3. Suppression des doublons
print("\nNombre de doublons avant suppression :", data.duplicated().sum())
data = data.drop_duplicates()
print("Nombre de doublons après suppression :", data.duplicated().sum())


Nombre de doublons avant suppression : 0
Nombre de doublons après suppression : 0


In [477]:
# 4. Nettoyage des chaînes de caractères (suppression des espaces inutiles, mise en minuscule)
def clean_text_columns(df, columns):
    for col in columns:
        df[col] = df[col].str.strip().str.lower()
    return df

text_columns = ['Nom', 'Genre', 'Ville', 'Tolérance au Risque', 'Historique d\'Investissement',
                 'Objectif Financier', 'Secteur Préféré', 'Fréquence d\'Investissement', 'Domaine Recommandé']
data = clean_text_columns(data, text_columns)

In [478]:
# 5. Harmonisation des genres
data['Genre'] = data['Genre'].replace({'homme': 'Homme', 'femme': 'Femme'})

In [479]:
# 6. Affichage des catégories uniques dans certaines colonnes pour vérification
print("\nCatégories uniques pour Genre :")
print(data['Genre'].unique())


Catégories uniques pour Genre :
['Femme' 'Homme']


In [480]:
print("\nCatégories uniques pour Tolérance au Risque :")
print(data['Tolérance au Risque'].unique())


Catégories uniques pour Tolérance au Risque :
['élevé' 'moyen' 'faible']


In [481]:
print("\nCatégories uniques pour Fréquence d'Investissement :")
print(data['Fréquence d\'Investissement'].unique())


Catégories uniques pour Fréquence d'Investissement :
['mensuel' 'trimestriel' 'annuel']


In [482]:
print("\nCatégories uniques pour Domaine Recommandé :")
print(data['Domaine Recommandé'].unique())


Catégories uniques pour Domaine Recommandé :
['actions' 'obligations' 'immobilier' 'etf' 'cryptomonnaies' 'startups']


In [483]:
# 7. Encodage optionnel (si nécessaire pour le Machine Learning)
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoded_columns = ['Genre', 'Tolérance au Risque', 'Objectif Financier', 'Secteur Préféré', 'Fréquence d\'Investissement', 'Domaine Recommandé']
for col in encoded_columns:
    data[col] = encoder.fit_transform(data[col])

In [484]:
# 8. Résumé statistique des colonnes numériques
print("\nRésumé statistique des colonnes numériques :")
print(data.describe())


Résumé statistique des colonnes numériques :
             Genre          Age         Revenu  Tolérance au Risque  \
count  1000.000000  1000.000000    1000.000000          1000.000000   
mean      0.487000    42.039000   61025.692030             1.323000   
std       0.500081    11.038357   28184.942734             0.488784   
min       0.000000    25.000000    5843.110000             0.000000   
25%       0.000000    34.000000   40586.727500             1.000000   
50%       0.000000    42.000000   56614.175000             1.000000   
75%       1.000000    50.000000   76320.660000             2.000000   
max       1.000000    70.000000  164195.870000             2.000000   

       Objectif Financier  Secteur Préféré  Fréquence d'Investissement  \
count         1000.000000       1000.00000                 1000.000000   
mean             2.619000          3.38900                    1.716000   
std              1.760949          1.80913                    0.582824   
min              0

In [485]:
# 1. Vérification des informations de base sur le dataset
print("\nInformations générales sur le dataset :")
print(data.info())


Informations générales sur le dataset :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Nom                          1000 non-null   object 
 1   Genre                        1000 non-null   int32  
 2   Ville                        1000 non-null   object 
 3   Age                          1000 non-null   int64  
 4   Revenu                       1000 non-null   float64
 5   Tolérance au Risque          1000 non-null   int32  
 6   Historique d'Investissement  1000 non-null   object 
 7   Objectif Financier           1000 non-null   int32  
 8   Secteur Préféré              1000 non-null   int32  
 9   Fréquence d'Investissement   1000 non-null   int32  
 10  Domaine Recommandé           1000 non-null   int32  
dtypes: float64(1), int32(6), int64(1), object(3)
memory usage: 62.6+ KB
None


In [486]:
# 1. Suppression de la colonne "Nom" (non utile pour le ML)
data = data.drop(columns=['Nom'])

# 2. Encodage des villes (Label Encoding)
data['Ville'] = encoder.fit_transform(data['Ville'])

In [487]:
# Liste des types d'investissements possibles
investment_types = set()
data['Historique d\'Investissement'].apply(lambda x: investment_types.update(x.split(',')))

# Nettoyer et ordonner les types d'investissement
investment_types = sorted([inv_type.strip().lower() for inv_type in investment_types])

# Encoder la colonne "Historique d'Investissement" en une chaîne binaire
def encode_investments(investment_string, investment_types):
    # Convertir en binaire selon les types d'investissement
    investment_string = investment_string.lower()
    return ''.join(['1' if inv_type in investment_string else '0' for inv_type in investment_types])

# Appliquer l'encodage dans la même colonne
data['Historique d\'Investissement'] = data['Historique d\'Investissement'].apply(lambda x: encode_investments(x, investment_types))

print("\nAperçu des données après encodage de 'Historique d'Investissement' :")
print(data.head())



Aperçu des données après encodage de 'Historique d'Investissement' :
   Genre  Ville  Age    Revenu  Tolérance au Risque  \
0      0      7   48  91061.68                    2   
1      1      4   60  95493.18                    1   
2      0      6   40  84344.87                    2   
3      0      2   44  59332.79                    1   
4      0      5   40  75818.07                    2   

  Historique d'Investissement  Objectif Financier  Secteur Préféré  \
0          110000000000001100                   2                5   
1          000011110000000000                   6                4   
2          000000001100001100                   2                2   
3          110000000000001100                   2                3   
4          000011111100000000                   0                5   

   Fréquence d'Investissement  Domaine Recommandé  
0                           1                   0  
1                           2                   4  
2                     

In [488]:
# 1. Vérification des informations de base sur le dataset
print("\nInformations générales sur le dataset :")
print(data.info())


Informations générales sur le dataset :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Genre                        1000 non-null   int32  
 1   Ville                        1000 non-null   int32  
 2   Age                          1000 non-null   int64  
 3   Revenu                       1000 non-null   float64
 4   Tolérance au Risque          1000 non-null   int32  
 5   Historique d'Investissement  1000 non-null   object 
 6   Objectif Financier           1000 non-null   int32  
 7   Secteur Préféré              1000 non-null   int32  
 8   Fréquence d'Investissement   1000 non-null   int32  
 9   Domaine Recommandé           1000 non-null   int32  
dtypes: float64(1), int32(7), int64(1), object(1)
memory usage: 50.9+ KB
None


In [489]:
# Encodage de la colonne cible avec LabelEncoder
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
data['Domaine Recommandé'] = encoder.fit_transform(data['Domaine Recommandé'])

# Affichage des correspondances (catégorie -> valeur numérique)
label_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print("Mapping des valeurs encodées :", label_mapping)


Mapping des valeurs encodées : {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}


In [490]:
# 9. Sauvegarde du dataset nettoyé
#cleaned_file_path = "newdata1.csv"
#data.to_csv(cleaned_file_path, index=False)
#print(f"\nDataset nettoyé sauvegardé sous : {cleaned_file_path}")

In [491]:
# Trouver les colonnes de type 'object'
object_columns = data.select_dtypes(include=['object']).columns

# Initialiser un dictionnaire pour sauvegarder les mappings
encoders = {}
encoded_data = data.copy()

# Encoder chaque colonne de type 'object'
for col in object_columns:
    encoder = LabelEncoder()
    encoded_data[col] = encoder.fit_transform(encoded_data[col])
    encoders[col] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))  # Sauvegarder le mapping pour chaque colonne

# Affichage des encodages réalisés
print("\nMappings des colonnes encodées :")
for col, mapping in encoders.items():
    print(f"\n{col} : {mapping}")
# Aperçu des données encodées
print("\nAperçu des premières lignes des données encodées :")
print(encoded_data.head())
# Sauvegarder le dataset encodé si besoin
#encoded_file_path = "encoded_user_investment_data.csv"
#encoded_data.to_csv(encoded_file_path, index=False)
#print(f"\nDataset encodé sauvegardé sous : {encoded_file_path}")



Mappings des colonnes encodées :

Historique d'Investissement : {'000000000000000011': 0, '000000000000001100': 1, '000000000000001111': 2, '000000000000011111': 3, '000000000000100000': 4, '000000000000100011': 5, '000000000000101100': 6, '000000000011001100': 7, '000000000011010000': 8, '000000000011100000': 9, '000000000011101100': 10, '000000001100000000': 11, '000000001100000011': 12, '000000001100001100': 13, '000000001100001111': 14, '000000001100011100': 15, '000000001100100000': 16, '000000001111000000': 17, '000000001111001100': 18, '000000110000000000': 19, '000000110000000011': 20, '000000110000001100': 21, '000000110000001111': 22, '000000110000100000': 23, '000000110011000000': 24, '000000111100000000': 25, '000000111100001100': 26, '000011000000000000': 27, '000011000000000011': 28, '000011000000001100': 29, '000011000000001111': 30, '000011000000011111': 31, '000011000000100000': 32, '000011000000101100': 33, '000011000011000000': 34, '000011000011000011': 35, '0000110

In [492]:
# 1. Vérification des informations de base sur le dataset
print("\nInformations générales sur le dataset :")
print(data.info())


Informations générales sur le dataset :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Genre                        1000 non-null   int32  
 1   Ville                        1000 non-null   int32  
 2   Age                          1000 non-null   int64  
 3   Revenu                       1000 non-null   float64
 4   Tolérance au Risque          1000 non-null   int32  
 5   Historique d'Investissement  1000 non-null   object 
 6   Objectif Financier           1000 non-null   int32  
 7   Secteur Préféré              1000 non-null   int32  
 8   Fréquence d'Investissement   1000 non-null   int32  
 9   Domaine Recommandé           1000 non-null   int64  
dtypes: float64(1), int32(6), int64(2), object(1)
memory usage: 54.8+ KB
None


In [493]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import warnings
from imblearn.over_sampling import SMOTE


# Ignorer les warnings
warnings.filterwarnings("ignore")

# Séparation des variables explicatives (X) et de la cible (y)
X = data.drop(columns=['Domaine Recommandé'])  # Features
y = data['Domaine Recommandé']  # Target

# Appliquer SMOTE pour équilibrer les classes
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

print(f"Nombre de lignes après équilibrage : {len(y_balanced)}")
print(f"Distribution des classes après SMOTE : {y_balanced.value_counts()}")

# Encoder la colonne "Historique d'Investissement"
encoder = LabelEncoder()
X['Historique d\'Investissement'] = encoder.fit_transform(X['Historique d\'Investissement'])

# Division en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(
    n_estimators=100,
    class_weight="balanced",  # Pondération automatique des classes
    random_state=42
)

rf_model.fit(X_train, y_train)

# Prédictions
y_pred = rf_model.predict(X_test)

# Évaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest - Précision : {accuracy * 100:.2f}%")


Nombre de lignes après équilibrage : 1824
Distribution des classes après SMOTE : Domaine Recommandé
0    304
4    304
3    304
2    304
1    304
5    304
Name: count, dtype: int64


Random Forest - Précision : 83.00%


In [494]:
import pickle

# Enregistrer le modèle entraîné
with open("lsi2.pkl", "wb") as f:
    pickle.dump(model, f)

print("Modèle enregistré dans 'lsi2.pkl'")


Modèle enregistré dans 'lsi2.pkl'


In [495]:
print(model.feature_names_in_)


['Genre' 'Ville' 'Age' 'Revenu' 'Tolérance au Risque'
 "Historique d'Investissement" 'Objectif Financier' 'Secteur Préféré'
 "Fréquence d'Investissement"]


In [496]:
from collections import Counter
print(Counter(y_train))  # y_train étant la liste des valeurs "Domaine Recommandé"


Counter({2: 241, 3: 225, 4: 167, 0: 150, 5: 9, 1: 8})


In [497]:
import pickle

with open("lsi2.pkl", "rb") as f:
    model = pickle.load(f)

# Liste de différentes entrées à tester
test_inputs = [
    [0, 1, 25, 30000, 2, 3, 1, 0, 2],  # Entrée 1 : autre profil
    [1, 2, 45, 100000, 5, 4, 2, 1, 5],  # Entrée 2 : profil plus risqué
    [1, 3, 30, 70000, 1, 1, 3, 2, 1],   # Entrée 3 : investisseur conservateur
]

for input_data in test_inputs:
    prediction = model.predict([input_data])[0]
    print(f"Entrée: {input_data} -> Prédiction: {prediction}")


Entrée: [0, 1, 25, 30000, 2, 3, 1, 0, 2] -> Prédiction: 5
Entrée: [1, 2, 45, 100000, 5, 4, 2, 1, 5] -> Prédiction: 5
Entrée: [1, 3, 30, 70000, 1, 1, 3, 2, 1] -> Prédiction: 5


In [499]:
print(y_train.value_counts())


Domaine Recommandé
2    241
3    225
4    167
0    150
5      9
1      8
Name: count, dtype: int64
